In [43]:
import pyrealsense2 as rs
import numpy as np
import cv2
import copy
import math
import os
import tifffile as tf

In [44]:
# read image
path = r'images'
filenames = os.listdir(path)
img = tf.imread(os.path.join(path, filenames[0]))

In [2]:
video_path = r"3D_videos_round1"
filelist = [file for file in os.listdir(video_path) if "under" in file]

In [84]:
# camera intrinsics
scale = 0.001
fx = 383.305 
fy = 383.305
cx = 319.135 
cy = 236.548

In [3]:
# get intrinsics object 
config = rs.config()
rs.config.enable_device_from_file(config, os.path.join(video_path, filelist[0]), repeat_playback = False)
pipeline = rs.pipeline()
cfg = pipeline.start(config) # Start pipeline and get the configuration it found
profile = cfg.get_stream(rs.stream.depth) # Fetch stream profile for depth stream
intr = profile.as_video_stream_profile().get_intrinsics() # Downcast to video_stream_profile and fetch intrinsics

In [108]:
# point location
ux = 50
uy = 120
vx = 60
vy = 250

In [109]:
# point distance from depth
udist = img[uy, ux] * scale
vdist = img[vy, vx] * scale
gdist = img[int(cy), int(cx)] * scale

In [110]:
# point x, y, dist 
point1 = rs.rs2_deproject_pixel_to_point(intr, [ux, uy], udist)
point2 = rs.rs2_deproject_pixel_to_point(intr, [vx, vy], vdist)
point3 = rs.rs2_deproject_pixel_to_point(intr, [int(cx), int(cy)], gdist)

In [111]:
point1

[-0.8264234662055969, -0.3578789234161377, 1.1770000457763672]

In [112]:
point2

[-0.7010690569877625, 0.03639407455921173, 1.0369999408721924]

In [105]:
point3

[-0.00021072618255857378, -0.0008544980082660913, 0.5979999899864197]

In [161]:
# intrinsic matrix
mat = [[fx, 0, cx], [0, fy, cy], [0,0,1]]
mat = np.array(mat)
out = np.matmul(np.linalg.inv(mat),(np.array([vx,vy,vdist])))
print(out)

[-0.70686006  0.01226106  1.037     ]


In [143]:
mat = [[fx, 0, cx], [0, fy, cy], [0,0,1]]
mat = np.array(mat)
out = np.matmul(udist*(np.array([ux-cx,uy-cy,1])),np.linalg.inv(mat))
out

array([ -0.82642255,  -0.35787949, 349.57303647])